# Model Training

In [1]:
import pandas as pd
df=pd.read_csv("data\gemstone.csv")
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [2]:
df=df.drop(labels=['id'],axis=1)

In [3]:
## Independent and dependent features
X = df.drop(columns=['price'],axis=1)
Y = df[['price']]

In [4]:
# Define which columns should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [6]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [8]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

An imputer is a tool or method that is used to fill in the missing values in a dataset. There are several techniques for imputing missing data, such as:

Mean imputation: Replace missing values with the mean value of the non-missing values in the same column.

Median imputation: Replace missing values with the median value of the non-missing values in the same column.

Mode imputation: Replace missing values with the mode value of the non-missing values in the same column.

Interpolation: Estimate the missing values based on the values of the neighboring data points in the same column.

Machine learning-based imputation: Train a machine learning model to predict the missing values based on the available data.

In [11]:
#Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
        ("Imputer", SimpleImputer(strategy="median")),
        ("Scalar", StandardScaler())
    ]
)

#Categorical Pipeline
cate_pipeline=Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("oridinal_encoder", OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories])),
        ("Scalar", StandardScaler())
    ]
)
#combine two pipelines
preprocessor=ColumnTransformer([
    
    ("num_pipeline", num_pipeline, numerical_cols),
    ("cate_columns", cate_pipeline, categorical_cols)
    
])

In [13]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.20,random_state=2)

In [14]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [16]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cate_columns__cut,cate_columns__color,cate_columns__clarity
0,-1.060445,0.905461,-0.639742,-1.312068,-1.306707,-1.239040,0.872088,-0.932872,2.015372
1,-0.152987,0.074634,0.923961,0.048945,0.063027,0.051505,-0.133675,-0.317168,-0.650593
2,0.451985,1.920917,-0.639742,0.544678,0.589150,0.762029,-2.145200,1.529946,-0.650593
3,-0.606716,1.367032,-1.160976,-0.591001,-0.562878,-0.456013,-2.145200,1.529946,-0.650593
4,-0.174593,-2.048591,2.487663,0.012892,-0.063968,-0.209504,-0.133675,0.298537,0.682390


In [17]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


In [18]:
regression=LinearRegression()
regression.fit(X_train, y_train)

LinearRegression()

In [19]:
regression.coef_

array([[ 6425.82545912,  -129.86886379,   -70.31594972, -1739.92234457,
         -434.2250013 ,   -97.82497178,    70.44192728,  -463.34951672,
          649.33294382]])

In [20]:
regression.intercept_

array([3970.27104186])

In [25]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae,mse, rmse, r2_square

In [26]:
y_pred=regression.predict(X_test)

In [27]:
evaluate_model(y_test, y_pred)

(675.4047722396335, 1030761.9750771999, 1015.2644852831206, 0.9366573147331785)

In [42]:
#Train Multiple Models 

models={
    "LinearRegression":LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "ElasticNet": ElasticNet()
}

model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train, y_train)
    
    y_pred=model.predict(X_test)
    
    mae, mse, rmse, r2_square=evaluate_model(y_test, y_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1015.2644852831206
MAE: 675.4047722396335
R2 score 93.66573147331785


Lasso
Model Training Performance
RMSE: 1015.3450734374022
MAE: 676.5496054583353
R2 score 93.66472584911601


Ridge
Model Training Performance
RMSE: 1015.2682505632112
MAE: 675.4349896180526
R2 score 93.66568448981799


ElasticNet
Model Training Performance
RMSE: 1542.4485316836347
MAE: 1068.1938210502663
R2 score 85.37959361883442




In [43]:
for i, j in zip(model_list, r2_list):
    print(i, j)

LinearRegression 0.9366573147331785
Lasso 0.93664725849116
Ridge 0.9366568448981799
ElasticNet 0.8537959361883443
